In [10]:
import torch
from torch import nn
from torchvision import datasets,transforms,models

import matplotlib.pyplot as plt
from pathlib import Path
from helper_functions import download_data
from engine import train
from data_setup import create_dataloaders
from utils import save_model
import pandas as pd
from timeit import default_timer as timer

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
effnetb2_file_path = Path("models/effnetb2_pizza_steak_sushi.pt")
effnetb2 = models.efficientnet_b2(weights=None)  

for param in effnetb2.parameters():
    param.requires_grad = False
effnetb2.classifier = nn.Sequential(
    nn.Dropout(p=0.3),
    nn.Linear(in_features=1408,out_features=3)
)
effnetb2.load_state_dict(torch.load(f=effnetb2_file_path))
effnetb2.to(device=device)
effnetb2_transforms = models.EfficientNet_B2_Weights.DEFAULT.transforms

In [11]:
data_path = Path("data/pizza_steak_sushi_20_percent/")
train_dir = data_path / "train"
test_dir = data_path / "test"

In [12]:
train_dataloader_effnetb2,test_dataloader_effnetb2,class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=effnetb2_transforms,
    batch_size=32
)

In [ ]:
class_names

['pizza', 'steak', 'sushi']

: 

In [ ]:
def predict(img,model):
    start_time = timer()
    
    img = effnetb2_transforms(img).unsqueeze(0)
    
    model.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(model(img), dim=1)
    
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    pred_time = round(timer() - start_time, 5)
    
    return pred_labels_and_probs, pred_time